In [ ]:
# PRACTICA 1 - Aprenentatge Supervisat
# AAMD - GEI 2024-25

# --------------------------------------------------------
# 🔹 0. Imports bàsics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import mean_absolute_percentage_error, accuracy_score, confusion_matrix, roc_curve, roc_auc_score


In [ ]:
# --------------------------------------------------------
# 🔹 1. Lectura de dades
df_train = pd.read_csv("diamonds-train.csv")
df_test = pd.read_csv("diamonds-test.csv")

print(df_train.shape)
df_train.head()


In [ ]:
# --------------------------------------------------------
# 🔹 2. Preprocessament
# a. Detecció de valors no físics
# b. Detecció d'outliers amb boxplots, histogrames, scatter plots

# c. Taula d'outliers amb raons i tractament (eliminació o correcció)
# d. Conversió d'atributs categòrics ordinals a valors numèrics

# e. Estandardització de les variables (excepte x,y,z,price)
# f. Escalat conjunt de x,y,z entre [-1, 1]
# g. Transformació logarítmica del preu + escalar entre [0.1, 0.9]
# h. Creació del nou atribut binari 'high_price'


In [ ]:
# --------------------------------------------------------
# 🔹 3. Predicció del preu (log_price escalat)
# Regressió Multilineal
# KNN
# Xarxa Neuronal (MLPRegressor)

# Cross-validation per cada model (usant GridSearchCV o validació manual)
# Càlcul de MAPE sobre conjunt de test

# Prediccions i scatter plots reals vs predits
# Conclusions


In [ ]:
# --------------------------------------------------------
# 🔹 4. Classificació del preu (high_price)
# Regressió logística
# SVM
# Xarxa neuronal (MLPClassifier)

# Cross-validation i avaluació (accuracy, sensitivity, specificity)
# ROC i AUC per LR i MLNN

# Prediccions al test set i comparació
# Conclusions finals
